In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib
%matplotlib inline

**This notebook contains the pickled models from Model for Defense notebook, and uses them to make predictions on players that the models were not trained on (played less than 5 seasons, or had to skip year 5 due to injury or taking a break, or incoming college players for the 2017 NBA Draft)**

In [35]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',10000)
df = pd.read_csv('/Users/mattlee/project-capstone/Clean Data/brdraft.csv')

In [5]:
df.head()

,Year,Pick,Team,Name,College,Seasons,Games,Minutes Played,Points,Total Rebounds,Assists,FG%,3P%,FT%,MPG,PPG,TRPG,APG,Win Share,Win Share/48,Box +/-,VORP
0,1991,1,CHH,Larry Johnson\johnsla02,University of Nevada Las Vegas,10.0,707.0,25685.0,11450.0,5300.0,2298.0,0.484,0.332,0.766,36.3,16.2,7.5,3.3,69.7,0.130,2.4,28.3
1,1991,2,NJN,Kenny Anderson\anderke01,Georgia Institute of Technology,14.0,858.0,25868.0,10789.0,2641.0,5196.0,0.421,0.346,0.790,30.1,12.6,3.1,6.1,62.5,0.116,0.8,18.0
2,1991,3,SAC,Billy Owens\owensbi01,Syracuse University,10.0,600.0,17619.0,7026.0,4016.0,1704.0,0.481,0.291,0.629,29.4,11.7,6.7,2.8,28.6,0.078,0.5,11.2
3,1991,4,DEN,Dikembe Mutombo\mutomdi01,Georgetown University,18.0,1196.0,36791.0,11729.0,12359.0,1240.0,0.518,0.000,0.684,30.8,9.8,10.3,1.0,117.0,0.153,2.1,38.2
4,1991,5,MIA,Steve Smith\smithst01,Michigan State University,14.0,942.0,28855.0,13430.0,3060.0,2922.0,0.440,0.358,0.845,30.6,14.3,3.2,3.1,83.7,0.139,1.4,24.4


In [65]:
df['Name'] = df['Name'].apply(lambda x: x.split('\\')[0].lower().title())

In [8]:
df.loc[df['Year'] == 2010]

,Year,Pick,Team,Name,College,Seasons,Games,Minutes Played,Points,Total Rebounds,Assists,FG%,3P%,FT%,MPG,PPG,TRPG,APG,Win Share,Win Share/48,Box +/-,VORP
1094,2010,1,WAS,John Wall,University of Kentucky,7.0,500.0,18031.0,9419.0,2218.0,4610.0,0.433,0.321,0.791,36.1,18.8,4.4,9.2,40.5,0.108,2.5,20.4
1095,2010,2,PHI,Evan Turner,Ohio State University,7.0,534.0,15048.0,5619.0,2668.0,1968.0,0.431,0.298,0.782,28.2,10.5,5.0,3.7,15.9,0.051,-2.0,-0.1
1096,2010,3,NJN,Derrick Favors,Georgia Institute of Technology,7.0,479.0,12348.0,5479.0,3435.0,510.0,0.510,0.154,0.664,25.8,11.4,7.2,1.1,33.5,0.130,0.8,8.7
1097,2010,4,MIN,Wesley Johnson,Syracuse University,7.0,497.0,11452.0,3704.0,1636.0,561.0,0.406,0.337,0.743,23.0,7.5,3.3,1.1,9.0,0.038,-1.2,2.3
1098,2010,5,SAC,Demarcus Cousins,University of Kentucky,7.0,487.0,15570.0,10308.0,5270.0,1468.0,0.459,0.330,0.736,32.0,21.2,10.8,3.0,36.9,0.114,2.6,18.1
1099,2010,6,GSW,Ekpe Udoh,Baylor University,5.0,270.0,4564.0,1072.0,860.0,179.0,0.429,0.000,0.720,16.9,4.0,3.2,0.7,5.7,0.060,-1.1,1.0
1100,2010,7,DET,Greg Monroe,Georgetown University,7.0,538.0,15955.0,7572.0,4707.0,1216.0,0.511,0.000,0.705,29.7,14.1,8.7,2.3,45.2,0.136,2.0,16.2
1101,2010,8,LAC,Al-Farouq Aminu,Wake Forest University,7.0,520.0,12520.0,3766.0,2948.0,639.0,0.426,0.323,0.725,24.1,7.2,5.7,1.2,17.4,0.067,-0.3,5.2
1102,2010,9,UTA,Gordon Hayward,Butler University,7.0,516.0,16164.0,8077.0,2148.0,1762.0,0.444,0.368,0.820,31.3,15.7,4.2,3.4,43.0,0.128,2.0,16.3
1103,2010,10,IND,Paul George,California State University Fresno,7.0,448.0,14692.0,8090.0,2816.0,1419.0,0.432,0.370,0.850,32.8,18.1,6.3,3.2,44.5,0.145,3.8,21.3


In [15]:
df1 = df[['Year','Pick','Team','Name','College']].copy()

In [16]:
college = pd.read_csv('/Users/mattlee/GA-Capstone-Project/FEdf1.csv')

In [20]:
college.head()

,Name,Team,PPG,Games Played,MPG,FGM,FGA,FG%,FTM,FTA,FT%,3PM,3PA,3P%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year,NBATeam,CollegeName,Age at Draft,Weight at Draft,Height Inches,PER,WS,WS/48,BPM,VORP,TopSchool,Total,TotalWS,TotalWS48,TotalPCAWS,TotalPCAWS48,TotalDef,TotalPERWS,TotalPERWSBPM,TotalWSBPM,TotalWSBPMV,PERVORP,TotalWV,MPG3,FTMSR,3PMSR,ORBSR,DRBL,APGCR,SPGSR,TOVSR,Age at Draft SR,Height Inches C,TotalWSSR,TotalPCAWSSR,TotalPERWSSR,TotalWSBPMSR,TotalWSBPMVCR,PERVORPCR,Games Played 4,TotalOff,FTASR,FT%C,3PASR,3P%R,RPGSR
0,A.J. Price,UConn,12.466667,33.000000,29.30,4.166667,10.10,0.410667,2.633333,3.600000,0.719000,1.50,4.166667,0.348,0.800,2.433333,3.233333,4.566667,1.066667,0.066667,2.300000,1.300000,2009.0,IND,University of Connecticut,22.5,193.0,74.0,11.66,4.4,0.0440,-2.98,-0.3,0,1.686755,1.154851,1.602797,-0.222778,-0.178794,0.672569,0.478970,1.016996,0.621984,0.759839,0.532868,0.739942,25153.757000,1.622755,1.224745,0.894427,0.889262,1.659077,1.032796,1.516575,506.25,405224.0,1.074640,NaN,0.692077,0.788659,0.912516,0.810724,1.185921e+06,1.731432,1.897367,0.371695,2.041241,1.000053,1.798147
1,Aaron Gray,PITT,8.750000,28.250000,18.40,3.425000,6.25,0.554250,1.900000,3.175000,0.590500,0.00,0.000000,0.000,2.275,3.975000,6.275000,1.175000,0.300000,1.025000,1.350000,1.975000,2007.0,CHI,University of Pittsburgh,22.3,280.0,85.0,12.82,5.5,0.0952,-1.32,0.3,0,1.966063,1.303480,1.865614,-0.155401,-0.050713,1.537938,0.542123,1.152496,0.710822,0.861807,0.592658,0.758726,6229.504000,1.378405,0.000000,1.508310,1.380025,1.055227,0.547723,1.161895,497.29,614125.0,1.141701,NaN,0.736290,0.843103,0.951634,0.839978,6.369038e+05,2.684290,1.781853,0.205902,0.000000,inf,2.504995
2,Al Horford,UF,9.866667,36.333333,25.50,3.766667,6.40,0.565333,2.433333,3.966667,0.612333,0.00,0.066667,0.000,2.300,5.500000,7.766667,1.633333,0.833333,1.700000,1.533333,2.433333,2007.0,ATL,University of Florida,20.8,246.0,82.0,18.16,34.6,0.1580,2.52,12.3,1,3.207373,2.384504,2.670641,0.411597,0.352678,1.949977,1.193208,1.970937,1.314461,1.728028,1.070043,0.989856,16581.375000,1.559915,0.000000,1.516575,1.704748,1.177674,0.912871,1.238278,432.64,551368.0,1.544184,0.641559,1.092341,1.146499,1.200006,1.022823,1.742693e+06,2.954779,1.991649,0.229596,0.258199,inf,2.786874
3,Al-Farouq Aminu,WF,13.300000,31.000000,30.15,4.650000,9.80,0.481500,3.600000,5.200000,0.684500,0.40,1.550000,0.226,3.050,5.650000,8.700000,1.300000,1.150000,1.250000,2.650000,2.550000,2010.0,LAC,Wake Forest University,19.5,216.0,80.0,12.30,11.6,0.0652,-0.36,3.5,0,2.071892,1.485879,1.879988,-0.058963,-0.045261,1.621651,0.612660,1.264872,0.844116,1.065122,0.641763,0.837860,27407.028375,1.897367,0.632456,1.746425,1.731656,1.091393,1.072381,1.627882,380.25,512000.0,1.218966,NaN,0.782726,0.918758,1.021253,0.862564,9.235210e+05,2.034810,2.280351,0.320716,1.244990,1.000074,2.949576
4,Alec Burks,UCOL,18.450000,34.000000,30.80,6.100000,12.20,0.503000,5.650000,7.100000,0.798500,0.65,2.150000,0.322,2.100,3.500000,5.650000,2.300000,1.150000,0.350000,2.200000,2.350000,2011.0,UTA,University of Colorado,19.7,193.0,78.0,13.72,8.9,0.0738,-2.10,0.2,0,1.962438,1.354475,1.811014,-0.119639,-0.072433,0.824115,0.629300,1.205679,0.727803,0.876599,0.626673,0.786119,29218.112000,2.376973,0.806226,1.449138,1.252763,1.320006,1.072381,1.483240,388.09,474552.0,1.163819,NaN,0.793285,0.853113,0.957048,0.855750,1.336336e+06,2.392888,2.664583,0.509125,1.466288,1.000057,2.376973


In [22]:
draft2010 = df1.loc[df1['Year'] == 2010].copy()

In [26]:
names = list(draft2010['Name'].unique())

In [27]:
college2010 = college.loc[college['Name'].isin(names)]

In [31]:
college2010.head()

,Name,Team,PPG,Games Played,MPG,FGM,FGA,FG%,FTM,FTA,FT%,3PM,3PA,3P%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year,NBATeam,CollegeName,Age at Draft,Weight at Draft,Height Inches,PER,WS,WS/48,BPM,VORP,TopSchool,Total,TotalWS,TotalWS48,TotalPCAWS,TotalPCAWS48,TotalDef,TotalPERWS,TotalPERWSBPM,TotalWSBPM,TotalWSBPMV,PERVORP,TotalWV,MPG3,FTMSR,3PMSR,ORBSR,DRBL,APGCR,SPGSR,TOVSR,Age at Draft SR,Height Inches C,TotalWSSR,TotalPCAWSSR,TotalPERWSSR,TotalWSBPMSR,TotalWSBPMVCR,PERVORPCR,Games Played 4,TotalOff,FTASR,FT%C,3PASR,3P%R,RPGSR
3,Al-Farouq Aminu,WF,13.300000,31.0,30.150000,4.650000,9.800000,0.481500,3.6,5.200000,0.684500,0.4,1.55,0.226,3.050000,5.650000,8.700000,1.300000,1.150000,1.250000,2.650000,2.550000,2010.0,LAC,Wake Forest University,19.5,216.0,80.0,12.30,11.6,0.0652,-0.36,3.5,0,2.071892,1.485879,1.879988,-0.058963,-0.045261,1.621651,0.612660,1.264872,0.844116,1.065122,0.641763,0.837860,27407.028375,1.897367,0.632456,1.746425,1.731656,1.091393,1.072381,1.627882,380.25,512000.0,1.218966,NaN,0.782726,0.918758,1.021253,0.862564,923521.0,2.034810,2.280351,0.320716,1.244990,1.000074,2.949576
16,Avery Bradley,UT,10.700000,34.0,29.500000,4.300000,9.900000,0.432000,1.0,1.800000,0.545000,1.2,3.00,0.381,0.900000,1.700000,2.600000,1.900000,1.200000,0.500000,1.400000,2.100000,2010.0,BOS,University of Texas at Austin,19.4,180.0,75.0,9.30,6.6,0.0106,-3.58,-0.3,0,1.513410,1.066754,1.396469,-0.257760,-0.285413,0.971347,0.417022,0.928898,0.628818,0.766673,0.437936,0.760745,25672.375000,1.000000,1.095445,0.948683,0.530628,1.238562,1.095445,1.183216,376.36,421875.0,1.032838,NaN,0.645772,0.792980,0.915244,0.759399,1336336.0,1.732320,1.341641,0.161879,1.732051,1.000048,1.612452
38,Cole Aldrich,KU,9.333333,37.0,21.533333,3.533333,6.133333,0.559333,2.3,3.133333,0.718333,0.0,0.00,0.000,2.233333,5.466667,7.733333,0.633333,0.533333,2.266667,1.166667,2.066667,2010.0,NOH,University of Kansas,21.4,236.0,83.0,14.60,4.8,0.1194,-0.48,0.9,0,2.138674,1.414325,2.048719,-0.107264,0.042730,1.735681,0.603229,1.250211,0.736937,0.901051,0.677388,0.760309,9984.671704,1.516575,0.000000,1.494434,1.698669,0.858771,0.730297,1.080123,457.96,571787.0,1.189254,NaN,0.776678,0.858450,0.965865,0.878239,1874161.0,2.473642,1.770122,0.370662,0.000000,inf,2.780887
60,Demarcus Cousins,UK,14.500000,38.0,23.500000,5.200000,9.300000,0.560000,4.1,6.700000,0.607000,0.0,0.20,0.167,3.900000,5.500000,9.400000,1.000000,0.900000,1.700000,2.000000,3.100000,2010.0,SAC,University of Kentucky,19.7,292.0,83.0,21.56,23.6,0.1056,1.74,10.2,1,2.965533,2.276406,2.593719,0.338036,0.330381,2.075797,1.165056,1.908791,1.115549,1.483164,1.160857,0.941411,12977.875000,2.024846,0.000000,1.974842,1.704748,1.000000,0.948683,1.414214,388.09,571787.0,1.508776,0.581409,1.079378,1.056196,1.140415,1.050976,2085136.0,3.243105,2.588436,0.223649,0.447214,1.000089,3.065942
63,Derrick Favors,GT,12.100000,36.0,27.500000,4.800000,7.900000,0.611000,2.4,3.800000,0.629000,0.0,0.00,0.000,2.900000,5.200000,8.200000,1.000000,0.900000,2.000000,2.400000,2.500000,2010.0,NJN,Georgia Institute of Technology,18.7,245.0,82.0,17.84,23.7,0.1200,0.26,5.9,0,2.695554,1.969673,2.322241,0.198969,0.186193,1.737759,1.016918,1.696150,1.052546,1.326069,0.917127,0.916556,20796.875000,1.549193,0.000000,1.702939,1.648659,1.000000,0.948683,1.549193,349.69,551368.0,1.403451,0.446059,1.008423,1.025937,1.098640,0.971575,1679616.0,3.045126,1.949359,0.248858,0.000000,inf,2.863564


**My training data only includes players who have played 5 years straight in the NBA, (e.g. Jordan Crawford played 5 seasons, but was not on a team after his 4th season, and rejoined 2 years later)**

In [53]:
draftcompare = draft2010.copy()

In [33]:
draftcompare = draftcompare.merge(college2010, on = 'Name')

In [36]:
draftcompare.head()

,Year_x,Pick,Name,Team_x,College,Team_y,PPG,Games Played,MPG,FGM,FGA,FG%,FTM,FTA,FT%,3PM,3PA,3P%,ORB,DRB,RPG,APG,SPG,BPG,TOV,PF,Year_y,NBATeam,CollegeName,Age at Draft,Weight at Draft,Height Inches,PER,WS,WS/48,BPM,VORP,TopSchool,Total,TotalWS,TotalWS48,TotalPCAWS,TotalPCAWS48,TotalDef,TotalPERWS,TotalPERWSBPM,TotalWSBPM,TotalWSBPMV,PERVORP,TotalWV,MPG3,FTMSR,3PMSR,ORBSR,DRBL,APGCR,SPGSR,TOVSR,Age at Draft SR,Height Inches C,TotalWSSR,TotalPCAWSSR,TotalPERWSSR,TotalWSBPMSR,TotalWSBPMVCR,PERVORPCR,Games Played 4,TotalOff,FTASR,FT%C,3PASR,3P%R,RPGSR
0,2010,1,John Wall,WAS,University of Kentucky,UK,15.900000,37.000000,34.8,5.200000,11.300000,0.461000,4.5,6.0,0.754,1.000000,3.0,0.325,0.800000,3.3,4.1,6.200000,1.700000,0.500000,3.8,1.900000,2010.0,WAS,University of Kentucky,19.6,196.0,76.0,18.74,25.9,0.1012,1.98,12.3,1,2.931762,2.253865,2.525465,0.330194,0.287130,1.387582,1.086104,1.840299,1.160492,1.574058,1.093374,0.961055,42144.192,2.121320,1.000000,0.894427,1.193922,1.837091,1.303840,1.949359,384.16,438976.0,1.501288,0.574625,1.042163,1.077261,1.163251,1.030203,1.874161e+06,2.470582,2.449490,0.428661,1.732051,1.000056,2.024846
1,2010,2,Evan Turner,PHI,Ohio State University,OHSU,16.133333,33.666667,33.1,5.733333,11.333333,0.497667,4.1,5.4,0.747,0.566667,1.5,0.379,1.566667,5.6,7.2,4.366667,1.666667,0.766667,3.7,2.833333,2010.0,PHI,Ohio State University,21.4,214.0,79.0,12.14,10.7,0.0488,-2.20,-0.6,0,1.840197,1.296042,1.661786,-0.142672,-0.149324,1.575607,0.592731,1.164751,0.750431,0.881722,0.545611,0.791022,36264.691,2.024846,0.752773,1.251666,1.722767,1.634494,1.290994,1.923538,457.96,493039.0,1.138438,NaN,0.769890,0.866274,0.958909,0.817136,1.284696e+06,1.694181,2.323790,0.416833,1.224745,1.000049,2.683282
2,2010,3,Derrick Favors,NJN,Georgia Institute of Technology,GT,12.100000,36.000000,27.5,4.800000,7.900000,0.611000,2.4,3.8,0.629,0.000000,0.0,0.000,2.900000,5.2,8.2,1.000000,0.900000,2.000000,2.4,2.500000,2010.0,NJN,Georgia Institute of Technology,18.7,245.0,82.0,17.84,23.7,0.1200,0.26,5.9,0,2.695554,1.969673,2.322241,0.198969,0.186193,1.737759,1.016918,1.696150,1.052546,1.326069,0.917127,0.916556,20796.875,1.549193,0.000000,1.702939,1.648659,1.000000,0.948683,1.549193,349.69,551368.0,1.403451,0.446059,1.008423,1.025937,1.098640,0.971575,1.679616e+06,3.045126,1.949359,0.248858,0.000000,inf,2.863564
3,2010,4,Wesley Johnson,MIN,Syracuse University,SU,15.700000,35.000000,35.0,5.600000,11.200000,0.502000,3.0,3.9,0.772,1.400000,3.3,0.415,2.000000,6.1,8.1,2.100000,1.600000,1.700000,2.2,2.000000,2010.0,MIN,Syracuse University,22.7,206.0,79.0,10.12,5.5,0.0258,-1.88,1.1,0,1.673423,1.187972,1.572974,-0.209494,-0.198984,1.031449,0.433515,1.019481,0.686416,0.854906,0.501555,0.769008,42875.000,1.732051,1.183216,1.414214,1.808289,1.280579,1.264911,1.483240,515.29,493039.0,1.089941,NaN,0.658419,0.828502,0.949087,0.794523,1.500625e+06,1.638402,1.974842,0.460100,1.816590,1.000044,2.846050
4,2010,5,Demarcus Cousins,SAC,University of Kentucky,UK,14.500000,38.000000,23.5,5.200000,9.300000,0.560000,4.1,6.7,0.607,0.000000,0.2,0.167,3.900000,5.5,9.4,1.000000,0.900000,1.700000,2.0,3.100000,2010.0,SAC,University of Kentucky,19.7,292.0,83.0,21.56,23.6,0.1056,1.74,10.2,1,2.965533,2.276406,2.593719,0.338036,0.330381,2.075797,1.165056,1.908791,1.115549,1.483164,1.160857,0.941411,12977.875,2.024846,0.000000,1.974842,1.704748,1.000000,0.948683,1.414214,388.09,571787.0,1.508776,0.581409,1.079378,1.056196,1.140415,1.050976,2.085136e+06,3.243105,2.588436,0.223649,0.447214,1.000089,3.065942


**Lets try defensive models first**

In [39]:
x = draftcompare[[u'MPG3', u'FG%', u'FTMSR', u'FT%C', u'3PMSR', u'DRBL', u'APGCR',
       u'SPGSR', u'TOVSR', u'BPG', u'PF', u'Age at Draft SR',
       u'Weight at Draft', u'Height Inches C', u'TopSchool']]

In [40]:
deflr = joblib.load('Defense_LR.pkl')
defrg = joblib.load('Defense_Ridge.pkl')
defls = joblib.load('Defense_LS.pkl')
defgbr = joblib.load('Defense_GBR.pkl')

In [42]:
predictionslr = deflr.predict(x)
predictionsrg = defrg.predict(x)
predictionsls = defls.predict(x)
predictionsgbr = defgbr.predict(x)


In [43]:
draftcompare['PredictionsLR'] = predictionslr
draftcompare['PredictionsRG'] = predictionsrg
draftcompare['PredictionsLS'] = predictionsls
draftcompare['PredictionsGBR'] = predictionsgbr

In [47]:
draftcompare2010 = draftcompare[['Year_x','Pick','Name','Team_x','College','TotalDef','PredictionsLR','PredictionsRG',
                                'PredictionsLS','PredictionsGBR']].copy()

In [51]:
draftcompare2010.sort_values(by = 'PredictionsLS',ascending = False)

,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
4,2010,5,Demarcus Cousins,SAC,University of Kentucky,2.075797,1.887773,1.894938,1.893847,1.799438
6,2010,7,Greg Monroe,DET,Georgetown University,1.803689,1.805676,1.795040,1.770437,1.583664
2,2010,3,Derrick Favors,NJN,Georgia Institute of Technology,1.737759,1.768672,1.746068,1.730688,1.678309
21,2010,33,Hassan Whiteside,SAC,Marshall University,1.270202,1.516268,1.535387,1.559494,1.428129
10,2010,11,Cole Aldrich,NOH,University of Kansas,1.735681,1.555675,1.555822,1.548478,1.554575
14,2010,15,Larry Sanders,MIL,Virginia Commonwealth University,1.794154,1.466408,1.482676,1.482810,1.637714
1,2010,2,Evan Turner,PHI,Ohio State University,1.575607,1.448145,1.444004,1.445264,1.639644
7,2010,8,Al-Farouq Aminu,LAC,Wake Forest University,1.621651,1.408893,1.420440,1.444027,1.492290
13,2010,14,Patrick Patterson,HOU,University of Kentucky,1.249155,1.472869,1.451486,1.405873,1.497201
12,2010,13,Ed Davis,TOR,University of North Carolina,1.684962,1.395890,1.396932,1.393863,1.574983


In [136]:
def getpredictdfdef(draftdf,featuredf,year,model1,model2,model3,model4):
    tomerge = draftdf[['Year','Pick','Team','Name','College']].copy()
    draftyear = tomerge.loc[tomerge['Year'] == year].copy()
    names = list(draftyear['Name'])
    collegedf = featuredf.loc[featuredf['Name'].isin(names)].copy()
    draftcomp = draftyear.merge(collegedf, on = 'Name')
    x = draftcomp[[u'MPG3', u'FG%', u'FTMSR', u'FT%C', u'3PMSR', u'DRBL', u'APGCR',
       u'SPGSR', u'TOVSR', u'BPG', u'PF', u'Age at Draft SR',
       u'Weight at Draft', u'Height Inches C', u'TopSchool']]
    predictlr = model1.predict(x)
    predictrg = model2.predict(x)
    predictls = model3.predict(x)
    predictgbr = model4.predict(x)
    draftcomp['PredictionsLR'] = predictlr
    draftcomp['PredictionsRG'] = predictrg
    draftcomp['PredictionsLS'] = predictls
    draftcomp['PredictionsGBR'] = predictgbr
    draftcompfinal = draftcomp[['Year_x','Pick','Name','Team_x','College','TotalDef','PredictionsLR','PredictionsRG',
                                'PredictionsLS','PredictionsGBR']].copy()
    draftcompfinal = draftcompfinal.sort_values(by = 'PredictionsLS', ascending = False)
    draftcompfinal = draftcompfinal.reset_index(drop = True)
    draftcompfinal = draftcompfinal.rename_axis('Model Pick')
    draftcompfinal = draftcompfinal.reset_index()
    draftcompfinal['Model Pick'] = draftcompfinal['Model Pick'] + 1
    return draftcompfinal

In [99]:
newplayer = pd.read_csv('/Users/mattlee/GA-Capstone-Project/newplayers.csv')

In [102]:
newplayer['Year'] = newplayer['Year_y']
newplayer['Age at Draft'] = newplayer['Age at Draft_y']
newplayer['Weight at Draft'] = newplayer['Weight at Draft_y']
newplayer['Height Inches'] = newplayer['Height Inches_y']

In [104]:
newplayer.to_csv('newplayers.csv', index = False)

In [137]:
d2010 = getpredictdfdef(df,college,2010,deflr,defrg,defls,defgbr)
d2010

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2010,5,Demarcus Cousins,SAC,University of Kentucky,2.075797,1.887773,1.894938,1.893847,1.799438
1,2,2010,7,Greg Monroe,DET,Georgetown University,1.803689,1.805676,1.795040,1.770437,1.583664
2,3,2010,3,Derrick Favors,NJN,Georgia Institute of Technology,1.737759,1.768672,1.746068,1.730688,1.678309
3,4,2010,33,Hassan Whiteside,SAC,Marshall University,1.270202,1.516268,1.535387,1.559494,1.428129
4,5,2010,11,Cole Aldrich,NOH,University of Kansas,1.735681,1.555675,1.555822,1.548478,1.554575
5,6,2010,15,Larry Sanders,MIL,Virginia Commonwealth University,1.794154,1.466408,1.482676,1.482810,1.637714
6,7,2010,2,Evan Turner,PHI,Ohio State University,1.575607,1.448145,1.444004,1.445264,1.639644
7,8,2010,8,Al-Farouq Aminu,LAC,Wake Forest University,1.621651,1.408893,1.420440,1.444027,1.492290
8,9,2010,14,Patrick Patterson,HOU,University of Kentucky,1.249155,1.472869,1.451486,1.405873,1.497201
9,10,2010,13,Ed Davis,TOR,University of North Carolina,1.684962,1.395890,1.396932,1.393863,1.574983


In [138]:
d2003 = getpredictdfdef(df,college,2003,deflr,defrg,defls,defgbr)
d2003

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2003,6,Chris Kaman,LAC,Central Michigan University,1.882391,1.598424,1.585758,1.599303,1.767628
1,2,2003,4,Chris Bosh,TOR,Georgia Institute of Technology,1.745957,1.483850,1.465349,1.438179,1.576599
2,3,2003,18,David West,NOH,Xavier University,1.583155,1.445442,1.451821,1.437389,1.556944
3,4,2003,5,Dwyane Wade,MIA,Marquette University,1.435503,1.431022,1.429383,1.434679,1.448667
4,5,2003,3,Carmelo Anthony,DEN,Syracuse University,1.192669,1.292539,1.298202,1.284089,1.246475
5,6,2003,24,Brian Cook,LAL,University of Illinois at Urbana-Champaign,1.080683,1.149096,1.162509,1.180430,1.109306
6,7,2003,32,Luke Walton,LAL,University of Arizona,1.143338,1.124097,1.133784,1.142667,0.990169
7,8,2003,29,Josh Howard,DAL,Wake Forest University,1.511106,1.126300,1.126548,1.140929,1.236930
8,9,2003,8,T.J. Ford,MIL,University of Texas at Austin,0.959581,1.082178,1.105642,1.127821,1.144355
9,10,2003,13,Marcus Banks,MEM,University of Nevada Las Vegas,0.818126,1.128655,1.105949,1.098100,1.130877


In [139]:
d2004 = getpredictdfdef(df,college,2004,deflr,defrg,defls,defgbr)
d2004

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2004,2,Emeka Okafor,CHA,University of Connecticut,2.004996,1.703221,1.695931,1.685842,1.590484
1,2,2004,43,Trevor Ariza,NYK,University of California Los Angeles,1.406483,1.504741,1.525220,1.548028,1.530798
2,3,2004,9,Andre Iguodala,PHI,University of Arizona,1.692594,1.386833,1.409093,1.430388,1.463149
3,4,2004,7,Luol Deng,PHO,Duke University,1.624614,1.320535,1.320885,1.320988,1.331545
4,5,2004,14,Kris Humphries,UTA,University of Minnesota,1.259956,1.182272,1.213804,1.279943,1.319820
5,6,2004,25,Tony Allen,BOS,Oklahoma State University,1.262130,1.226622,1.228272,1.233996,1.469322
6,7,2004,24,Delonte West,BOS,Saint Joseph's University,1.162673,1.087324,1.064603,1.048041,1.071726
7,8,2004,20,Jameer Nelson,DEN,Saint Joseph's University,1.071303,1.026433,1.011918,1.004228,0.967339
8,9,2004,38,Chris Duhon,CHI,Duke University,1.079547,0.984836,0.990887,1.000275,1.045751
9,10,2004,5,Devin Harris,WAS,University of Wisconsin,1.036585,1.058364,1.045503,0.998658,1.106913


**Solid defensive picks**

In [140]:
d2005 = getpredictdfdef(df,college,2004,deflr,defrg,defls,defgbr)
d2005

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2004,2,Emeka Okafor,CHA,University of Connecticut,2.004996,1.703221,1.695931,1.685842,1.590484
1,2,2004,43,Trevor Ariza,NYK,University of California Los Angeles,1.406483,1.504741,1.525220,1.548028,1.530798
2,3,2004,9,Andre Iguodala,PHI,University of Arizona,1.692594,1.386833,1.409093,1.430388,1.463149
3,4,2004,7,Luol Deng,PHO,Duke University,1.624614,1.320535,1.320885,1.320988,1.331545
4,5,2004,14,Kris Humphries,UTA,University of Minnesota,1.259956,1.182272,1.213804,1.279943,1.319820
5,6,2004,25,Tony Allen,BOS,Oklahoma State University,1.262130,1.226622,1.228272,1.233996,1.469322
6,7,2004,24,Delonte West,BOS,Saint Joseph's University,1.162673,1.087324,1.064603,1.048041,1.071726
7,8,2004,20,Jameer Nelson,DEN,Saint Joseph's University,1.071303,1.026433,1.011918,1.004228,0.967339
8,9,2004,38,Chris Duhon,CHI,Duke University,1.079547,0.984836,0.990887,1.000275,1.045751
9,10,2004,5,Devin Harris,WAS,University of Wisconsin,1.036585,1.058364,1.045503,0.998658,1.106913


In [141]:
d2006 = getpredictdfdef(df,college,2006,deflr,defrg,defls,defgbr)
d2006

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2006,4,Tyrus Thomas,POR,Louisiana State University,1.926476,1.831539,1.809305,1.768758,1.677235
1,2,2006,2,Lamarcus Aldridge,CHI,University of Texas at Austin,1.530630,1.758551,1.736074,1.696935,1.543350
2,3,2006,21,Rajon Rondo,PHO,University of Kentucky,1.790322,1.615632,1.599242,1.561922,1.542706
3,4,2006,47,Paul Millsap,UTA,Louisiana Tech University,1.813010,1.557910,1.547663,1.532309,1.730413
4,5,2006,5,Shelden Williams,ATL,Duke University,1.424349,1.502863,1.500979,1.511192,1.432598
5,6,2006,33,Solomon Jones,ATL,University of South Florida,1.120816,1.421681,1.443093,1.475303,1.244684
6,7,2006,36,Craig Smith,MIN,Boston College,1.145888,1.426628,1.413296,1.390172,1.433360
7,8,2006,17,Shawne Williams,IND,University of Memphis,0.972076,1.281922,1.309646,1.344978,1.204724
8,9,2006,20,Renaldo Balkman,NYK,University of South Carolina,1.456557,1.389707,1.369433,1.340772,1.393754
9,10,2006,8,Rudy Gay,HOU,University of Connecticut,1.304427,1.270751,1.281113,1.294460,1.253614


In [142]:
d2007 = getpredictdfdef(df,college,2007,deflr,defrg,defls,defgbr)
d2007

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2007,3,Al Horford,ATL,University of Florida,1.949977,1.844686,1.838214,1.801207,1.786966
1,2,2007,9,Joakim Noah,CHI,University of Florida,2.110336,1.738725,1.713994,1.668562,1.559627
2,3,2007,37,Josh Mcroberts,POR,Duke University,1.523931,1.688345,1.675676,1.630778,1.605698
3,4,2007,10,Spencer Hawes,SAC,University of Washington,1.585955,1.556784,1.564104,1.578167,1.460005
4,5,2007,8,Brandan Wright,CHA,University of North Carolina,1.165435,1.594675,1.550220,1.486401,1.480382
5,6,2007,17,Sean Williams,NJN,Boston College,1.316555,1.514048,1.505212,1.475241,1.489027
6,7,2007,20,Jason Smith,MIA,Colorado State University,1.226949,1.466235,1.468016,1.469657,1.317151
7,8,2007,35,Glen Davis,SEA,Louisiana State University,1.423925,1.417781,1.430435,1.454182,1.399191
8,9,2007,47,Dominic Mcguire,WAS,California State University Fresno,1.418592,1.428057,1.440560,1.449218,1.438597
9,10,2007,49,Aaron Gray,CHI,University of Pittsburgh,1.537938,1.443068,1.441035,1.427545,1.504515


In [143]:
d2008 = getpredictdfdef(df,college,2008,deflr,defrg,defls,defgbr)
d2008

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2008,5,Kevin Love,MEM,University of California Los Angeles,1.814360,1.611399,1.592411,1.558574,1.653143
1,2,2008,37,Luc Mbah A Moute,MIL,University of California Los Angeles,1.533023,1.554467,1.554234,1.550891,1.478689
2,3,2008,14,Anthony Randolph,GSW,Louisiana State University,1.434770,1.479538,1.509189,1.548643,1.372106
3,4,2008,19,J.J. Hickson,CLE,North Carolina State University,1.494843,1.546780,1.532889,1.542206,1.500254
4,5,2008,17,Roy Hibbert,TOR,Georgetown University,1.779346,1.574418,1.564344,1.532258,1.524035
5,6,2008,12,Jason Thompson,SAC,Rider University,1.347035,1.454384,1.461069,1.455741,1.424665
6,7,2008,2,Michael Beasley,MIA,Kansas State University,1.245729,1.454012,1.445445,1.451445,1.415862
7,8,2008,16,Marreese Speights,PHI,University of Florida,1.301872,1.523588,1.485956,1.446007,1.483303
8,9,2008,10,Brook Lopez,NJN,Stanford University,1.251440,1.370110,1.401052,1.439252,1.363857
9,10,2008,35,Deandre Jordan,LAC,Texas A&M University,1.744925,1.473697,1.447157,1.420532,1.727236


In [144]:
d2009 = getpredictdfdef(df,college,2009,deflr,defrg,defls,defgbr)
d2009

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2009,37,Dejuan Blair,SAS,University of Pittsburgh,1.698759,1.825332,1.816922,1.805752,1.782461
1,2,2009,2,Hasheem Thabeet,MEM,University of Connecticut,1.373288,1.510674,1.508716,1.499904,1.518034
2,3,2009,24,Byron Mullens,DAL,Ohio State University,1.005049,1.500381,1.470190,1.451165,1.573711
3,4,2009,8,Jordan Hill,NYK,University of Arizona,1.334899,1.477686,1.458217,1.432973,1.561940
4,5,2009,17,Jrue Holiday,PHI,University of California Los Angeles,1.151366,1.433763,1.435975,1.431425,1.169330
5,6,2009,4,Tyreke Evans,SAC,University of Memphis,1.123283,1.390682,1.395088,1.424223,1.440063
6,7,2009,14,Earl Clark,PHO,University of Louisville,1.223296,1.364291,1.370947,1.382878,1.309976
7,8,2009,16,James Johnson,CHI,Wake Forest University,1.417201,1.379633,1.372938,1.373724,1.510384
8,9,2009,26,Taj Gibson,CHI,University of Southern California,1.833233,1.391956,1.385024,1.366068,1.623047
9,10,2009,33,Dante Cunningham,POR,Villanova University,1.422233,1.312038,1.324391,1.341382,1.224709


In [155]:
d2011 = getpredictdfdef(df,college,2011,deflr,defrg,defls,defgbr)
d2011.sort_values(by = 'TotalDef',ascending = False)

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
1,2,2011,15,Kawhi Leonard,IND,San Diego State University,2.012758,1.560588,1.574898,1.585901,1.648864
2,3,2011,16,Nikola Vucevic,PHI,University of Southern California,1.840700,1.465424,1.462388,1.452030,1.320439
0,1,2011,22,Kenneth Faried,DEN,Morehead State University,1.704961,1.780078,1.771798,1.761931,1.898247
3,4,2011,4,Tristan Thompson,CLE,University of Texas at Austin,1.526085,1.464634,1.468735,1.441254,1.463267
4,5,2011,50,Lavoy Allen,PHI,Temple University,1.518908,1.401304,1.391886,1.340765,1.452013
6,7,2011,13,Markieff Morris,PHO,University of Kansas,1.490748,1.304929,1.295265,1.276403,1.256798
11,12,2011,30,Jimmy Butler,CHI,Marquette University,1.369184,1.116315,1.114661,1.086281,1.085194
7,8,2011,19,Tobias Harris,CHA,University of Tennessee,1.309181,1.241287,1.255907,1.270410,1.370091
5,6,2011,38,Chandler Parsons,HOU,University of Florida,1.303368,1.354143,1.349565,1.313733,1.254573
13,14,2011,40,Jon Leuer,MIL,University of Wisconsin,1.280863,1.098333,1.099764,1.080901,1.226887


In [156]:
d2012 = getpredictdfdef(df,college,2012,deflr,defrg,defls,defgbr)
d2012.sort_values(by = 'TotalDef', ascending = False)

,Model Pick,Year_x,Pick,Name,Team_x,College,TotalDef,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
1,2,2012,9,Andre Drummond,DET,University of Connecticut,2.397835,1.794366,1.800016,1.801870,1.549470
6,7,2012,35,Draymond Green,GSW,Michigan State University,2.252990,1.511152,1.507896,1.482811,1.661606
0,1,2012,1,Anthony Davis,NOH,University of Kentucky,2.121031,2.110061,2.082691,1.991552,1.835688
12,13,2012,49,Kyle O'Quinn,ORL,Norfolk State University,1.800597,1.336608,1.331071,1.318769,1.395032
7,8,2012,14,John Henson,MIL,University of North Carolina,1.644532,1.467250,1.477161,1.476747,1.534559
9,10,2012,5,Thomas Robinson,SAC,University of Kansas,1.598323,1.438383,1.434575,1.429767,1.557823
4,5,2012,21,Jared Sullinger,BOS,Ohio State University,1.541244,1.523064,1.525007,1.517218,1.470418
3,4,2012,2,Michael Kidd-Gilchrist,CHA,University of Kentucky,1.505237,1.538454,1.549083,1.555628,1.489015
2,3,2012,18,Terrence Jones,HOU,University of Kentucky,1.433071,1.595174,1.611581,1.610873,1.467128
15,16,2012,26,Miles Plumlee,IND,Duke University,1.403013,1.218510,1.215165,1.216917,1.335671


**Now we need to input new data to the model and see our predictions**

In [147]:
def getpredictnewdef(draftdf,featuredf,year,model1,model2,model3,model4):
    tomerge = draftdf[['Year','Pick','Team','Name','College']].copy()
    draftyear = tomerge.loc[tomerge['Year'] == year].copy()
    names = list(draftyear['Name'])
    collegedf = featuredf.loc[featuredf['Name'].isin(names)].copy()
    draftcomp = draftyear.merge(collegedf, on = 'Name')
    x = draftcomp[[u'MPG3', u'FG%', u'FTMSR', u'FT%C', u'3PMSR', u'DRBL', u'APGCR',
       u'SPGSR', u'TOVSR', u'BPG', u'PF', u'Age at Draft SR',
       u'Weight at Draft', u'Height Inches C', u'TopSchool']]
    predictlr = model1.predict(x)
    predictrg = model2.predict(x)
    predictls = model3.predict(x)
    predictgbr = model4.predict(x)
    draftcomp['PredictionsLR'] = predictlr
    draftcomp['PredictionsRG'] = predictrg
    draftcomp['PredictionsLS'] = predictls
    draftcomp['PredictionsGBR'] = predictgbr
    draftcompfinal = draftcomp[['Year_x','Pick','Name','Team_x','PredictionsLR','PredictionsRG',
                                'PredictionsLS','PredictionsGBR']].copy()
    draftcompfinal = draftcompfinal.sort_values(by = 'PredictionsLS', ascending = False)
    draftcompfinal = draftcompfinal.reset_index(drop = True)
    draftcompfinal = draftcompfinal.rename_axis('Model Pick')
    draftcompfinal = draftcompfinal.reset_index()
    draftcompfinal['Model Pick'] = draftcompfinal['Model Pick'] + 1
    return draftcompfinal

In [148]:
d2013 = getpredictnewdef(df,newplayer,2013,deflr,defrg,defls,defgbr)
d2013

,Model Pick,Year_x,Pick,Name,Team_x,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2013,6,Nerlens Noel,NOH,2.749987,2.736004,2.658599,1.938584
1,2,2013,21,Gorgui Dieng,UTA,2.268335,2.263168,2.219023,1.676681
2,3,2013,22,Mason Plumlee,BRK,2.181439,2.176375,2.137123,1.737067
3,4,2013,12,Steven Adams,OKC,2.173603,2.161751,2.117140,1.635583
4,5,2013,4,Cody Zeller,CHA,2.156083,2.143835,2.104681,1.599098
5,6,2013,5,Alex Len,PHO,1.976025,1.984357,1.965810,1.487819
6,7,2013,3,Otto Porter,WAS,2.025243,2.013069,1.941371,1.497579
7,8,2013,39,Jeff Withey,POR,1.905266,1.895959,1.861762,1.422998
8,9,2013,11,Michael Carter-Williams,PHI,1.816382,1.829510,1.812854,1.450348
9,10,2013,2,Victor Oladipo,ORL,1.826396,1.805100,1.774892,1.274577


In [149]:
d2014 = getpredictnewdef(df,newplayer,2014,deflr,defrg,defls,defgbr)
d2014

,Model Pick,Year_x,Pick,Name,Team_x,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2014,3,Joel Embiid,PHI,2.376027,2.354394,2.303524,1.764845
1,2,2014,30,Kyle Anderson,SAS,2.310856,2.321525,2.302164,1.640984
2,3,2014,7,Julius Randle,LAL,1.973079,1.989446,1.999910,1.465746
3,4,2014,4,Aaron Gordon,ORL,2.013344,2.010722,1.938391,1.637066
4,5,2014,9,Noah Vonleh,CHH,1.929634,1.928312,1.930446,1.356614
5,6,2014,6,Marcus Smart,BOS,1.856253,1.874224,1.879600,1.329858
6,7,2014,50,Alec Brown,PHO,1.767461,1.791697,1.801619,1.259719
7,8,2014,22,Jordan Adams,MEM,1.828386,1.828748,1.801350,1.005499
8,9,2014,2,Jabari Parker,MIL,1.768634,1.778761,1.788273,1.344793
9,10,2014,49,Cameron Bairstow,CHI,1.774164,1.789501,1.787392,1.291077


In [150]:
d2015 = getpredictnewdef(df,newplayer,2015,deflr,defrg,defls,defgbr)
d2015

,Model Pick,Year_x,Pick,Name,Team_x,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2015,6,Willie Cauley-Stein,SAC,2.440950,2.423632,2.353096,1.712457
1,2,2015,1,Karl-Anthony Towns,MIN,2.194729,2.193048,2.155404,1.618182
2,3,2015,56,Branden Dawson,NOP,2.169860,2.150218,2.117073,1.725861
3,4,2015,3,Jahlil Okafor,PHI,2.223198,2.176472,2.114169,1.733853
4,5,2015,29,Chris Mccullough,BRK,2.083769,2.100266,2.101066,1.506845
5,6,2015,30,Kevon Looney,GSW,2.103483,2.114898,2.061078,1.669565
6,7,2015,33,Jordan Mickey,BOS,1.971198,1.985897,2.006831,1.379389
7,8,2015,20,Delon Wright,TOR,2.055748,2.029379,1.958767,1.533383
8,9,2015,36,Rakeem Christmas,MIN,1.976581,1.973180,1.940268,1.515317
9,10,2015,12,Trey Lyles,UTA,1.890157,1.904096,1.900496,1.254273


In [151]:
d2016 = getpredictnewdef(df,newplayer,2016,deflr,defrg,defls,defgbr)
d2016

,Model Pick,Year_x,Pick,Name,Team_x,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,2016,1,Ben Simmons,PHI,2.467681,2.457267,2.403874,1.966407
1,2,2016,37,Chinanu Onuaku,HOU,2.273762,2.241696,2.183335,1.727236
2,3,2016,9,Jakob Poeltl,TOR,2.250065,2.208102,2.115499,1.733853
3,4,2016,25,Brice Johnson,LAC,2.129089,2.120204,2.071587,1.517658
4,5,2016,41,Stephen Zimmerman,ORL,1.997843,2.020867,2.022824,1.518622
5,6,2016,46,A.J. Hammons,DAL,1.985066,1.995580,2.006404,1.408746
6,7,2016,31,Deyonta Davis,BOS,2.057746,2.036547,1.968290,1.577629
7,8,2016,56,Daniel Hamilton,DEN,1.925761,1.948278,1.938491,1.334544
8,9,2016,30,Damian Jones,GSW,1.954845,1.957023,1.930406,1.652432
9,10,2016,5,Kris Dunn,MIN,1.883451,1.904613,1.924362,1.331847


**We will get the predictions for the 2017 NBA Draft**

In [125]:
earlyentry = pd.read_csv('/Users/mattlee/GA-Capstone-Project/earlyentrydf.csv')

In [153]:
def getpredictearlydef(df,model1,model2,model3,model4):
    earlydf = df
    x = df[[u'MPG3', u'FG%', u'FTMSR', u'FT%C', u'3PMSR', u'DRBL', u'APGCR',
       u'SPGSR', u'TOVSR', u'BPG', u'PF', u'Age at Draft SR',
       u'Weight at Draft', u'Height Inches C', u'TopSchool']]
    predictlr = model1.predict(x)
    predictrg = model2.predict(x)
    predictls = model3.predict(x)
    predictgbr = model4.predict(x)
    earlydf['PredictionsLR'] = predictlr
    earlydf['PredictionsRG'] = predictrg
    earlydf['PredictionsLS'] = predictls
    earlydf['PredictionsGBR'] = predictgbr
    earlydf = earlydf.sort_values(by = 'PredictionsLS', ascending = False)
    earlydf = earlydf.reset_index(drop = True)
    earlydf = earlydf.rename_axis('Model Pick')
    earlydf = earlydf.reset_index()
    earlydf['Model Pick'] = earlydf['Model Pick'] + 1
    return earlydf[['Model Pick','Name','Team','PredictionsLR','PredictionsRG','PredictionsLS','PredictionsGBR']]

In [134]:
earlyentryedit = pd.read_csv('/Users/mattlee/GA-Capstone-Project/earlyentrydf1.csv')

In [154]:
d2017 = getpredictearlydef(earlyentryedit,deflr,defrg,defls,defgbr)
d2017

,Model Pick,Name,Team,PredictionsLR,PredictionsRG,PredictionsLS,PredictionsGBR
0,1,Tacko Fall,UCF,2.436848,2.377179,2.295436,1.784391
1,2,Justin Patton,CU,2.251647,2.188036,2.070089,1.704461
2,3,Drew Eubanks,OSU,2.025960,2.016663,1.999796,1.556603
3,4,Jonathan Isaac,FSU,2.013604,2.007477,1.967949,1.221629
4,5,Josh Jackson,KU,2.045177,2.027523,1.967795,1.704601
5,6,Jarrett Allen,UT,1.981672,1.974633,1.963342,1.642300
6,7,Ivan Rabb,Cal,2.009082,2.003923,1.952168,1.517764
7,8,Lonzo Ball,UCLA,2.137149,2.080677,1.950275,1.500709
8,9,Caleb Swanigan,PU,1.887175,1.885508,1.880217,1.485655
9,10,Zach Collins,GU,1.985396,1.942578,1.878404,1.520818
